In [2]:
%cd mmdetection

# !pip install -q --upgrade wandb
# !pip3 install openmim
# !mim install mmcv-full
# !pip install -e .

from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.coco import CocoDataset,CustomDataset
from mmdet.apis import set_random_seed
import os.path as osp
import os
import numpy as np

import wandb
print(wandb.__version__)

/aiffel/aiffel/aimmo/mmdetection


/opt/conda/lib/python3.9/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


0.13.6


In [3]:
@DATASETS.register_module(force=True)
class Aimmo(CustomDataset):
    CLASSES = ('car', 'truck', 'bus','pedestrian')
    def load_annotations(self, ann_file):
        cat2label = {k:i for i, k in enumerate(self.CLASSES)}
        image_list = mmcv.list_from_file(self.ann_file)
        data_infos = []
    
        for image_id in image_list:
            filename = os.path.splitext(image_id)[0]#os.path.splitext('{0:}/{1:}'.format(img_prefix, image_id))[0]
            #image = cv2.imread(filename)
            data_info = {'filename': filename,
                         'width': 1920, 
                         'height': 1024}
            label_prefix = self.img_prefix.replace('image', 'middle_texts')
            lines = mmcv.list_from_file(osp.join(label_prefix, str(image_id)))

            content = [line.strip().split(' ') for line in lines]
            bbox_names = [x[0] for x in content]
            bboxes = [ [float(info) for info in x[1:5]] for x in content]

            gt_bboxes = []
            gt_labels = []
            gt_bboxes_ignore = []
            gt_labels_ignore = []

            for bbox_name, bbox in zip(bbox_names, bboxes):
                if bbox_name in cat2label:
                    gt_bboxes.append(bbox)
                    gt_labels.append(cat2label[bbox_name])
                else:
                    gt_bboxes_ignore.append(bbox)
                    gt_labels_ignore.append(-1)
            
            data_anno = {
              'bboxes': np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
              'labels': np.array(gt_labels, dtype=np.compat.long),
              'bboxes_ignore': np.array(gt_bboxes_ignore, dtype=np.float32).reshape(-1, 4),
              'labels_ignore': np.array(gt_labels_ignore, dtype=np.compat.long)
            }
            data_info.update(ann=data_anno)
            data_infos.append(data_info)
        return data_infos

In [4]:
config_file = 'configs/fcos/fcos_center-normbbox-centeronreg-giou_r50_caffe_fpn_gn-head_dcn_1x_coco.py'
checkpoint_file = 'fcos_center-normbbox-centeronreg-giou_r50_caffe_fpn_gn-head_dcn_1x_coco-ae4d8b3d.pth'
cfg = Config.fromfile(config_file)

cfg.load_from = 'checkpoints/fcos_center-normbbox-centeronreg-giou_r50_caffe_fpn_gn-head_dcn_1x_coco-ae4d8b3d.pth'
cfg.work_dir = 'fcos'
cfg.resume_from = 'fcos/epoch_7.pth'



cfg.dataset_type = 'Aimmo'
cfg.data_root = 'data/'

cfg.train_pipeline[2]['img_scale'] = (960,512)
cfg.test_pipeline[1]['img_scale'] = (960,512)



cfg.data.train.type = 'Aimmo'
cfg.data.train.data_root = 'data/'
cfg.data.train.ann_file = 'middle_train.txt'
cfg.data.train.img_prefix = 'image'

cfg.data.train.pipeline[2]['img_scale'] = (960,512)


cfg.data.val.type = 'Aimmo'
cfg.data.val.data_root = 'data/'
cfg.data.val.ann_file = 'middle_val.txt'
cfg.data.val.img_prefix = 'image'

cfg.data.val.pipeline[1]['img_scale'] = (960,512)

cfg.data.test.type = 'Aimmo'
cfg.data.test.data_root = 'data/'
cfg.data.test.ann_file = 'middle_sunday_test.txt'
cfg.data.test.img_prefix = 'test/image'

cfg.data.test.pipeline[1]['img_scale'] = (960,512)

cfg.lr_config.policy='step'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device='cuda'
cfg.evaluation.metric='mAP'


cfg.checkpoint_config.interval = 1
cfg.log_config.interval = 100
cfg.evaluation.interval = 1
cfg.runner['max_epochs'] = 15

cfg.model.bbox_head['num_classes'] =4

cfg.data.samples_per_gpu = 2
cfg.data.workers_per_gpu = 0

print(cfg.pretty_text)

dataset_type = 'Aimmo'
data_root = 'data/'
img_norm_cfg = dict(
    mean=[103.53, 116.28, 123.675], std=[1.0, 1.0, 1.0], to_rgb=False)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='Resize', img_scale=(960, 512), keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='Normalize',
        mean=[103.53, 116.28, 123.675],
        std=[1.0, 1.0, 1.0],
        to_rgb=False),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(960, 512),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(
                type='Normalize',
                mean=[103.53, 116.28, 123.675],
                std=[1.

In [5]:
from mmdet.apis import multi_gpu_test, single_gpu_test
from mmcv.parallel import MMDataParallel, MMDistributedDataParallel
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)


checkpoint_file = 'fcos/epoch_15.pth'  # 학습한 weight file
model_ckpt = init_detector(cfg, checkpoint_file, device='cuda:0')
model_ckpt = MMDataParallel(model_ckpt, device_ids=[0])

dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        # 반드시 아래 samples_per_gpu 인자값은 1로 설정
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

load checkpoint from local path: fcos/epoch_15.pth


2022-12-11 22:38:34,384 - root - INFO - ModulatedDeformConvPack backbone.layer2.0.conv2 is upgraded to version 2.
2022-12-11 22:38:34,386 - root - INFO - ModulatedDeformConvPack backbone.layer2.1.conv2 is upgraded to version 2.
2022-12-11 22:38:34,388 - root - INFO - ModulatedDeformConvPack backbone.layer2.2.conv2 is upgraded to version 2.
2022-12-11 22:38:34,390 - root - INFO - ModulatedDeformConvPack backbone.layer2.3.conv2 is upgraded to version 2.
2022-12-11 22:38:34,392 - root - INFO - ModulatedDeformConvPack backbone.layer3.0.conv2 is upgraded to version 2.
2022-12-11 22:38:34,395 - root - INFO - ModulatedDeformConvPack backbone.layer3.1.conv2 is upgraded to version 2.
2022-12-11 22:38:34,398 - root - INFO - ModulatedDeformConvPack backbone.layer3.2.conv2 is upgraded to version 2.
2022-12-11 22:38:34,401 - root - INFO - ModulatedDeformConvPack backbone.layer3.3.conv2 is upgraded to version 2.
2022-12-11 22:38:34,403 - root - INFO - ModulatedDeformConvPack backbone.layer3.4.conv2 

In [6]:
outputs = single_gpu_test(model_ckpt, data_loader, True, 'my_result', 0.3)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 393/393, 1.8 task/s, elapsed: 214s, ETA:     0s

In [7]:
dataset.evaluate(outputs,metric='mAP')


---------------iou_thr: 0.5---------------

+------------+------+-------+--------+-------+
| class      | gts  | dets  | recall | ap    |
+------------+------+-------+--------+-------+
| car        | 2057 | 12056 | 0.925  | 0.873 |
| truck      | 911  | 3573  | 0.472  | 0.078 |
| bus        | 319  | 8955  | 0.781  | 0.044 |
| pedestrian | 549  | 6918  | 0.727  | 0.516 |
+------------+------+-------+--------+-------+
| mAP        |      |       |        | 0.378 |
+------------+------+-------+--------+-------+


OrderedDict([('AP50', 0.378), ('mAP', 0.37784138321876526)])